## Detekcija gibanja

## Detekcija gibanja - slika razlik

In [26]:
import numpy as np
import cv2

capture = cv2.VideoCapture(0)

prviFrame = None

while True:
    ret, frame = capture.read()
    frameColor = frame
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.GaussianBlur(frame, (21, 21), 0)
    if prviFrame is None:
        prviFrame = frame
        continue
        
    if cv2.waitKey(10) & 0xFF == ord("q"):
        capture.release()
        cv2.destroyAllWindows()
        break
    
    
    razlika = cv2.absdiff(prviFrame,frame)
    thresh = cv2.threshold(razlika, 25, 255, cv2.THRESH_BINARY)[1]
    
    cv2.imshow("Original",frame)
    cv2.imshow("Razlika", razlika)
    cv2.imshow("Thresh", thresh)
    cv2.waitKey(1)

    
    thresh = cv2.dilate(thresh, None, iterations=2)
    contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) != 0:    
        c_max = max(contours, key=cv2.contourArea)
        x,y,w,h = cv2.boundingRect(c_max)
        cv2.rectangle(frameColor,(x,y),(x+w,y+h),(0,255,0),2)
        #cv2.drawContours(frameColor, contours, i, (255, 0, 0), 1)
    cv2.imshow("Color", frameColor)
    

## Detekcija gibanja - opencv variante

In [35]:
import cv2
capture = cv2.VideoCapture(0)

backSub = cv2.createBackgroundSubtractorMOG2()

#backSub = cv2.createBackgroundSubtractorKNN()
while True:
    ret, frame = capture.read()
    if frame is None:
        break
    
    fgMask = backSub.apply(frame)    
    
    thresh = cv2.threshold(fgMask, 25, 255, cv2.THRESH_BINARY)[1]
    contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) != 0:    
        c_max = max(contours, key=cv2.contourArea)
        x,y,w,h = cv2.boundingRect(c_max)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

    cv2.imshow('Frame', frame)
    cv2.imshow('FG Mask', fgMask)
    
    #keyboard = cv2.waitKey(30)
    if cv2.waitKey(300) & 0xFF == ord("q"):
        cv2.destroyAllWindows()
        capture.release()
        break
        

In [38]:
import cv2
import numpy as np
events = [i for i in dir(cv2) if 'EVENT' in i]
print(events)
global xLZ
global yLZ
global xLast
global yLast
global frame
xLZ = 0
yLZ = 0
frame = None
def clickOnImage(event, x, y, flags, param):
    global frame, xLZ, yLZ, xLast, yLast
    if event == cv2.EVENT_LBUTTONUP:
        #print("EVENT_LBUTTONUP ({},{})".format(x,y))
        xLZ = 0
        yLZ = 0
        
    if event == cv2.EVENT_LBUTTONDOWN:
        #print("EVENT_LBUTTONDOWN ({},{})".format(x,y))
        xLZ = x
        yLZ = y
        
    if event == cv2.EVENT_MOUSEMOVE:
        #print("EVENT_MOUSEMOVE ({},{})".format(x,y))
        xLast = x
        yLast = y       
        
capture = cv2.VideoCapture(0)
cv2.namedWindow("Slika")
cv2.setMouseCallback("Slika", clickOnImage)
miniFrame = None
track_window = None
while(1):
    ret, frame = capture.read()
    if ret == True:
        if xLZ != 0 or yLZ != 0:
            cv2.rectangle(frame,(xLZ,yLZ),(xLast,yLast),(0,255,0),2)
            
        cv2.imshow('Slika',frame)
        if cv2.waitKey(100) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
            capture.release()
            break
        if cv2.waitKey(100) & 0xFF == ord("r"):
            miniFrame = frame[yLZ:yLast,xLZ:xLast]
            track_window = (xLZ, yLZ, xLast-xLZ, yLast-yLZ)
            cv2.imshow("Mini",miniFrame)

if miniFrame is not None:    
    
    print("Is not none")
    roi = miniFrame
    hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))

    
    roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
    cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
    
    term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )
    
    capture = cv2.VideoCapture(0)
    while(1):
        ret, frame = capture.read()
        frame= cv2.flip(frame,1)
        if ret == True:
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
            # apply meanshift to get the new location
            ret, track_window = cv2.meanShift(dst, track_window, term_crit)
            #ret, track_window = cv2.CamShift(dst, track_window, term_crit)
            # Draw it on image
            x,y,w,h = track_window
            img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), 255,2)
            cv2.imshow('img2',img2)
            k = cv2.waitKey(30) & 0xff
            if k == 27:
                break
        else:
            break
    cv2.destroyAllWindows()
    capture.release()

['EVENT_FLAG_ALTKEY', 'EVENT_FLAG_CTRLKEY', 'EVENT_FLAG_LBUTTON', 'EVENT_FLAG_MBUTTON', 'EVENT_FLAG_RBUTTON', 'EVENT_FLAG_SHIFTKEY', 'EVENT_LBUTTONDBLCLK', 'EVENT_LBUTTONDOWN', 'EVENT_LBUTTONUP', 'EVENT_MBUTTONDBLCLK', 'EVENT_MBUTTONDOWN', 'EVENT_MBUTTONUP', 'EVENT_MOUSEHWHEEL', 'EVENT_MOUSEMOVE', 'EVENT_MOUSEWHEEL', 'EVENT_RBUTTONDBLCLK', 'EVENT_RBUTTONDOWN', 'EVENT_RBUTTONUP']


In [15]:
import cv2
import numpy as np

img = np.ones((3,3))
img[1,1] = 2
img[0,2] = 2
print(img)

mom = cv2.moments(img)
print(mom)
print (mom['m00'])
print (mom['m01'])
print (mom['m10'])

[[1. 1. 2.]
 [1. 2. 1.]
 [1. 1. 1.]]
{'m00': 11.0, 'm10': 12.0, 'm01': 10.0, 'm20': 20.0, 'm11': 10.0, 'm02': 16.0, 'm30': 36.0, 'm21': 16.0, 'm12': 16.0, 'm03': 28.0, 'mu20': 6.90909090909091, 'mu11': -0.9090909090909101, 'mu02': 6.909090909090908, 'mu30': -0.8925619834710758, 'mu21': -0.1983471074380132, 'mu12': 0.19834710743801853, 'mu03': 0.8925619834710687, 'nu20': 0.05709992486851992, 'nu11': -0.007513148009015786, 'nu02': 0.057099924868519905, 'nu30': -0.002224112097151733, 'nu21': -0.000494247132700376, 'nu12': 0.0004942471327003893, 'nu03': 0.0022241120971517154}
11.0
10.0
12.0
